In [52]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import numpy as np

In [5]:
gameRanks=pd.read_csv("GameRanks.csv")
gameIds= gameRanks['ID']

In [30]:
rootpath = 'https://boardgamegeek.com/xmlapi2/'
gamepath = 'thing/'
playpath = 'plays/'

In [37]:
gameparameters = {'id':gameIds[34],
              'stats' : 1}

playparameters = {"id": gameIds[34],
                  'type': 'thing',
                  'mindate':'2024-02-25'}

In [57]:
unrankedgames = gameRanks[gameRanks['Overall Rank'].isna()]['ID']

In [58]:
unrankedgames

100      7854
101    163412
102    287954
103    249259
104     31999
        ...  
746     15369
747    325348
748    303551
749    215565
750     67600
Name: ID, Length: 651, dtype: int64